# PROCESO DE ETL DEL DATASET movies_dataset.csv

In [1]:
#cargamos nuestras librerías que necitaremos
import pandas as pd
import ast
import numpy as np
from pandas.io.json import json_normalize


In [19]:
# Cargar los datos de ambos archivos Excel
df1 = pd.read_excel('..\datasets_crudos\movies_dataset.xlsx')
df2 = pd.read_excel('..\datasets_crudos\movies_dataset (1).xlsx')



C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_15576\2694318523.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df1 = pd.read_excel('..\datasets_crudos\movies_dataset.xlsx')
C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_15576\2694318523.py:3: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df2 = pd.read_excel('..\datasets_crudos\movies_dataset (1).xlsx')


In [20]:
# Comparar los datos en ambos dataframes
if df1.equals(df2):
    print("Los archivos tienen el mismo contenido")
else:
    print("Los archivos tienen contenido diferente")

Los archivos tienen el mismo contenido


In [24]:
#Al tener los dataframes el mismo contenido, guardamos uno a tipo csv
df1.to_csv('../datasets_crudos_CSV/movies_dataset_crudo.csv', index=False)
print("Se ha guardado df1 como un archivo CSV en datasets_crudos_csv.")

Se ha guardado df1 como un archivo CSV en datasets_crudos_csv.


In [26]:
'''En nuestra carpeta Datasets_crudos_csv verificamos que tenemos 2 archivos que pueden ser
que tenga la misma cantidad, vamos a averiguar si eso es cierto para descartar con cual dataset vamos a trabajar,
vamos a cargar nuestros datos y despues haremos la comparación'''
dfx = pd.read_csv('..\datasets_crudos_CSV\movies_dataset_crudo.csv')
dfy = pd.read_csv('..\datasets_crudos_CSV\movies_dataset.csv')

C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_15576\2023090911.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfx = pd.read_csv('..\datasets_crudos_CSV\movies_dataset_crudo.csv')
C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_15576\2023090911.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dfy = pd.read_csv('..\datasets_crudos_CSV\movies_dataset.csv')


In [27]:
#comparamos ambos datasets si contienen el mismo contenido
if dfx.equals(dfy):
    print("Los archivos tienen el mismo contenido")
else:
    print("Los archivos tienen contenido diferente")

Los archivos tienen contenido diferente


## Cargamos nuestro dataset en un dataframe de pandas para empezar el proceso de limpieza

In [2]:
'''Ahora que hemos verificado que los datos contienen contenido diferente, vamos a proceder
a explorar el contenido de los datos para proceder a realizar la limpieza
1.Primero el data de movies_dataser.csv'''
df_movies = pd.read_csv('..\datasets_crudos_CSV\movies_and_belong.csv')
df_movies.head()

,belongs_to_collection,id
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",862
1,NaN,8844
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",15602
3,NaN,31357
4,"{'id': 96871, 'name': 'Father of the Bride Col...",11862


In [3]:
'''Observamos que hay algunas columnas anidadas, asi que vamos a crear
una iteración para identificar esas columnas'''

nested_columns = ['belongs_to_collection', 'genres', 'spoken_languages', 'production_companies', 'production_countries']



In [4]:
df_movies =df_movies.rename(columns={'id': 'movies_id'})  # Renombrar la columna 'id' a 'movies_id'


In [5]:
df_movies.head(2)

,belongs_to_collection,movies_id
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",862
1,NaN,8844


In [6]:
null_counts = df_movies.isnull().sum()
print(null_counts)


belongs_to_collection    40972
movies_id                    0
dtype: int64


In [7]:
data_type = df_movies['movies_id'].dtypes
print(data_type)


object


In [8]:
'''creamos una función para extraer todos los datos de la lista 
de diccionarios de la columna belong_to_collection, para luego
separarlos en un dataframe aparte'''
def desanidar_belongs_to_collection(row):
    try:
        collection_dict = ast.literal_eval(row)
        if isinstance(collection_dict, dict):
            return (
                collection_dict.get('id'),
                collection_dict.get('name'),
                collection_dict.get('poster_path'),
                collection_dict.get('backdrop_path')
            )
    except (ValueError, SyntaxError):
        pass
    return (np.nan, np.nan, np.nan, np.nan)



In [9]:
df_movies[['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path']] = df_movies['belongs_to_collection'].apply(desanidar_belongs_to_collection).apply(pd.Series)


In [10]:
df_movies = df_movies.drop('belongs_to_collection', axis=1)


In [11]:
df_movies.head(2)

,movies_id,collection_id,collection_name,collection_poster_path,collection_backdrop_path
0,862,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,8844,NaN,NaN,NaN,NaN


In [12]:
null_counts = df_movies.isnull().sum()
print(null_counts)


movies_id                       0
collection_id               40975
collection_name             40975
collection_poster_path      41518
collection_backdrop_path    42203
dtype: int64


In [13]:
movies_with_names = df_movies.loc[~df_movies['collection_name'].isnull(), ['movies_id', 'collection_id', 'collection_name']]


In [14]:
movies_with_names_unique = movies_with_names.drop_duplicates()


In [15]:
print(movies_with_names_unique)



      movies_id  collection_id                 collection_name
0           862        10194.0            Toy Story Collection
2         15602       119050.0       Grumpy Old Men Collection
4         11862        96871.0  Father of the Bride Collection
9           710          645.0           James Bond Collection
12        21032       117693.0                Balto Collection
...         ...            ...                             ...
45355     24568        37261.0         The Carry On Collection
45358     19307        37261.0         The Carry On Collection
45369     21251        37261.0         The Carry On Collection
45371    460135       477208.0  DC Super Hero Girls Collection
45382    182030       200641.0            Red Lotus Collection

[4488 rows x 3 columns]


In [16]:
def buscar_por_id(id):
    return df_movies[df_movies['movies_id'] == id]


In [17]:
resultado = buscar_por_id(19307)
print(resultado)

Empty DataFrame
Columns: [movies_id, collection_id, collection_name, collection_poster_path, collection_backdrop_path]
Index: []


In [18]:
#guardamos nuestro nuevo dataset con el nombre de df_btc
movies_with_names_unique.to_csv('..\datasets_crudos_CSV\df_btc.csv', index=False)


## Desanidamos la conlumna genres

In [19]:
##cargamos nuestros datos
genres_desanidados = pd.read_csv('..\datasets_crudos_CSV\movies_and_genres.csv')

In [20]:
genres_desanidados.head()

,genres,id,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,NaN,NaN,NaN
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,NaN,NaN,NaN
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,NaN,NaN,NaN
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,NaN,NaN,NaN
4,"[{'id': 35, 'name': 'Comedy'}]",11862,NaN,NaN,NaN


In [21]:
genres_desanidados =genres_desanidados.rename(columns={'id': 'movies_id'}) 

In [22]:
data_type = genres_desanidados['movies_id'].dtypes
print(data_type)

object


In [23]:
genres_desanidados.head(2)

,genres,movies_id,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,NaN,NaN,NaN
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,NaN,NaN,NaN


In [24]:
columns_to_drop = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']
genres_desanidados = genres_desanidados.drop(columns=columns_to_drop)


In [25]:
genres_desanidados.head(2)

,genres,movies_id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844


In [26]:
#cargamos solo los datos que vamos a usar y trabajar
genres_desanidados = genres_desanidados[['genres', 'movies_id']]


In [27]:
genres_desanidados.head(3)

,genres,movies_id
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602


In [28]:


# Definir función para extraer los valores de la columna "genres"
def desanidar_genres(row):
    genres_list = ast.literal_eval(row)  # Convertir la cadena a una lista de diccionarios
    genres_dict = {}  # Diccionario para almacenar los géneros únicos
    for genre in genres_list:
        genre_id = genre['id']
        genre_name = genre['name']
        # Agregar el género al diccionario si aún no existe
        if genre_id not in genres_dict:
            genres_dict[genre_id] = genre_name
    # Crear una lista de géneros ordenada alfabéticamente
    genres = sorted(genres_dict.values())
    return genres


In [29]:
# Aplicar la función a la columna "genres"
genres_desanidados['genres'] = genres_desanidados['genres'].apply(desanidar_genres)

In [30]:
genres_desanidados.head()

,genres,movies_id
0,"[Animation, Comedy, Family]",862
1,"[Adventure, Family, Fantasy]",8844
2,"[Comedy, Romance]",15602
3,"[Comedy, Drama, Romance]",31357
4,[Comedy],11862


In [31]:
#guardamos nuestro nuevo dataset con el nombre de df_genres.csv
genres_desanidados.to_csv('..\datasets_crudos_CSV\df_genres.csv', index=False)

## Desanidamos la columna spoken_languages

In [32]:
languages = pd.read_csv('..\datasets_crudos_CSV\movies_languages.csv')

In [33]:
languages = languages.rename(columns={'id': 'movies_id'}) 

In [34]:
languages.head(2)

,movies_id,spoken_languages
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."


In [35]:
print(type(languages['spoken_languages'][1]))


<class 'str'>


In [36]:
# Definir una función para desanidar la columna spoken_languages
def desanidar_spoken_languages(row):
    if isinstance(row, str):
        try:
            spoken_languages = ast.literal_eval(row)
            iso_639_1 = []
            name = []
            if isinstance(spoken_languages, list):  # Agregar esta verificación para manejar el caso de una lista vacía
                for lang in spoken_languages:
                    iso_639_1.append(lang.get('iso_639_1'))
                    name.append(lang.get('name'))
            return iso_639_1, name
        except (ValueError, SyntaxError):
            pass
    elif isinstance(row, int):  # Manejar el caso de un valor entero
        return [], []
    return [], []




In [37]:
# Aplicar la función de desanidación a la columna spoken_languages
languages[['iso_639_1', 'name']] = languages['spoken_languages'].apply(desanidar_spoken_languages).apply(pd.Series)


In [38]:
#verificamos nuestro dataframe
languages.head()


,movies_id,spoken_languages,iso_639_1,name
0,862,"[{'iso_639_1': 'en', 'name': 'English'}]",[en],[English]
1,8844,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...","[en, fr]","[English, Français]"
2,15602,"[{'iso_639_1': 'en', 'name': 'English'}]",[en],[English]
3,31357,"[{'iso_639_1': 'en', 'name': 'English'}]",[en],[English]
4,11862,"[{'iso_639_1': 'en', 'name': 'English'}]",[en],[English]


In [39]:
#eliminamos la columna spoken_languages	
languages = languages.drop('spoken_languages', axis=1)
print(languages.head())


  movies_id iso_639_1                 name
0       862      [en]            [English]
1      8844  [en, fr]  [English, Français]
2     15602      [en]            [English]
3     31357      [en]            [English]
4     11862      [en]            [English]


In [40]:
#guardamos nuestro dataframe
languages.to_csv('..\datasets_crudos_CSV\df_languages.csv', index=False)

## Vamos unir nuestro dataframes de df_btc, df_genres, df_languages, con movies_dataset.csv

In [41]:
# Cargar el dataset movies_dataset.csv que es el original
movies_dataset = pd.read_csv('..\datasets_crudos_CSV\movies_dataset.csv')

C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_11040\217173532.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_dataset = pd.read_csv('..\datasets_crudos_CSV\movies_dataset.csv')


In [42]:
# Renombrar la columna "id" a "movies_id"
movies_dataset = movies_dataset.rename(columns={'id': 'movies_id'})

In [43]:
#eliminamos la columna genres para que no ocurra problema cuando unamos nuestro dataframe
movies_dataset = movies_dataset.drop('genres', axis=1)


In [44]:
'''En el dataset principal, cuando estaba haciendo la exploración de los datos
en la columna movies_id, observado que hay datos que tienen un formato de tipo fecha
por lo cual puede generar error cuando se quiera unir los dataframes, asi que puse averiguar 
si había mas de una'''
movies_dataset[movies_dataset['movies_id'] == '1997-08-20']

,adult,belongs_to_collection,budget,homepage,movies_id,imdb_id,original_language,original_title,overview,popularity,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
#corraboro que hay mas datos de ese tipo
rows_with_format = movies_dataset[movies_dataset['movies_id'].str.contains(r'\d{4}-\d{2}-\d{2}')]
print(rows_with_format)

                                                   adult  \
19730                                 - Written by Ørnås   
29503   Rune Balot goes to a casino connected to the ...   
35587   Avalanche Sharks tells the story of a bikini ...   

      belongs_to_collection                            budget  \
19730              0.065736  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   
29503              1.931659  /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg   
35587              2.185485  /zaSf5OG7V8X8gqFvly88zDdRm46.jpg   

                                                homepage   movies_id imdb_id  \
19730  [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...  1997-08-20       0   
29503  [{'iso_3166_1': 'US', 'name': 'United States o...  2012-09-29       0   
35587           [{'iso_3166_1': 'CA', 'name': 'Canada'}]  2014-01-01       0   

      original_language                            original_title  overview  \
19730             104.0  [{'iso_639_1': 'en', 'name': 'English'}]  Released   
29503              

In [46]:
#así que decido eliminarlo ya que la mayoría de los valores de las demas columnas son nulas
movies_dataset = movies_dataset[~movies_dataset['movies_id'].str.contains(r'\d{4}-\d{2}-\d{2}')]


In [47]:
'''siguiendo ahora buscando irregularidades en las columnas movies_id,
Los valores 'adv=no', '1997-08-20', '2012-09-29' y '2014-01-01' 
aparecen en las columnas 'movies_id' de los dataframes 'genres_desanidados' y 'languages'.'''
print(movies_dataset[~movies_dataset['movies_id'].astype(str).apply(str.isnumeric)]['movies_id'].unique())
print(movies_with_names_unique[~movies_with_names_unique['movies_id'].astype(str).apply(str.isnumeric)]['movies_id'].unique())
print(genres_desanidados[~genres_desanidados['movies_id'].astype(str).apply(str.isnumeric)]['movies_id'].unique())
print(languages[~languages['movies_id'].astype(str).apply(str.isnumeric)]['movies_id'].unique())


[]
[]
['adv=no' '1997-08-20' '2012-09-29' '2014-01-01']
['adv=no' '1997-08-20' '2012-09-29' '2014-01-01']


In [48]:
print(genres_desanidados[genres_desanidados['movies_id'].isin(['adv=no', '1997-08-20', '2012-09-29', '2014-01-01'])])
print(languages[languages['movies_id'].isin(['adv=no', '1997-08-20', '2012-09-29', '2014-01-01'])])


                                                  genres   movies_id
3211                           [Drama, Foreign, Romance]      adv=no
19730  [Carousel Productions, Telescene Film Group Pr...  1997-08-20
29503  [Aniplex, BROSTA TV, GoHands, Mardock Scramble...  2012-09-29
35587  [Odyssey Media, Pulser Productions, Rogue Stat...  2014-01-01
        movies_id iso_639_1 name
3211       adv=no        []   []
19730  1997-08-20        []   []
29503  2012-09-29        []   []
35587  2014-01-01        []   []


In [49]:
#decido eliminarlos
genres_desanidados = genres_desanidados[~genres_desanidados['movies_id'].isin(['adv=no', '1997-08-20', '2012-09-29', '2014-01-01'])]
languages = languages[~languages['movies_id'].isin(['adv=no', '1997-08-20', '2012-09-29', '2014-01-01'])]


In [50]:
'''convertir el tipo de datos de la columna movies_id en cada dataframe para que coincidan. 
Puedes hacerlo utilizando el método astype()'''
movies_dataset['movies_id'] = movies_dataset['movies_id'].astype(int)
movies_with_names_unique['movies_id'] = movies_with_names_unique['movies_id'].astype(int)
genres_desanidados['movies_id'] = genres_desanidados['movies_id'].astype(int)
languages['movies_id'] = languages['movies_id'].astype(int)


C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_11040\2302811060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_with_names_unique['movies_id'] = movies_with_names_unique['movies_id'].astype(int)


In [51]:
# Unir los dataframes utilizando merge
merged_df = movies_dataset.merge(movies_with_names_unique, on='movies_id', how='left')
merged_df = merged_df.merge(genres_desanidados, on='movies_id', how='left')
merged_df = merged_df.merge(languages, on='movies_id', how='left')

In [52]:
#verificamos la union
merged_df.head(2)

,adult,belongs_to_collection,budget,homepage,movies_id,imdb_id,original_language,original_title,overview,popularity,...,tagline,title,video,vote_average,vote_count,collection_id,collection_name,genres,iso_639_1,name
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,"[Animation, Comedy, Family]",[en],[English]
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,"[Adventure, Family, Fantasy]","[en, fr]","[English, Français]"


## Vamos eliminando por el momento las colunmnas que hemos desanidado

In [53]:
#Elimamos esatas columnas por el momento
columns_to_drop = ['belongs_to_collection', 'spoken_languages']
merged_df = merged_df.drop(columns=columns_to_drop)


In [54]:
'''Como nos piden en el requerimiento de que hay algunas columnas que no se van a usar como
video,imdb_id,adult,original_title,poster_path y homepage
vamos a proceder a eliminarlos de dataframe'''

columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
merged_df = merged_df.drop(columns_to_drop, axis=1)


In [55]:
merged_df.head()

,budget,movies_id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_id,collection_name,genres,iso_639_1,name
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,"[Animation, Comedy, Family]",[en],[English]
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,NaN,"[Adventure, Family, Fantasy]","[en, fr]","[English, Français]"
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,"[Comedy, Romance]",[en],[English]
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,NaN,"[Comedy, Drama, Romance]",[en],[English]
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,[Comedy],[en],[English]


In [56]:
'''Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0'''
merged_df['revenue'] = merged_df['revenue'].fillna(0)
merged_df['budget'] = merged_df['budget'].fillna(0)


In [57]:
'''Los valores nulos del campo release date deben eliminarse.'''
merged_df.dropna(subset=['release_date'], inplace=True)


In [58]:
'''De haber fechas, deberán tener el formato AAAA-mm-dd, 
además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.'''

# Convertir la columna "release_date" al tipo de dato datetime
merged_df['release_date'] = pd.to_datetime(merged_df['release_date'], errors='coerce')


In [59]:
# Crear la columna "release_year" con el año de la fecha de estreno
merged_df['release_year'] = merged_df['release_date'].dt.year

In [60]:
'''Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, 
dividiendo estas dos últimas revenue / budget, 
cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.'''

# Convertir las columnas "revenue" y "budget" a tipo numérico (float)
merged_df['revenue'] = pd.to_numeric(merged_df['revenue'], errors='coerce')
merged_df['budget'] = pd.to_numeric(merged_df['budget'], errors='coerce')

In [61]:
# Crear la columna "return" con el retorno de inversión
merged_df['return'] = merged_df['revenue'].div(merged_df['budget']).fillna(0)

In [62]:
# Convertir el tipo de dato a float
merged_df['return'] = merged_df['return'].astype(float)

In [63]:
#ordenamos nuestras columnas
column_order = ['movies_id'] + [col for col in merged_df.columns if col != 'movies_id']
merged_df = merged_df[column_order]


In [64]:
merged_df.head()

,movies_id,budget,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,title,vote_average,vote_count,collection_id,collection_name,genres,iso_639_1,name,release_year,return
0,862,30000000,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,...,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,"[Animation, Comedy, Family]",[en],[English],1995,12.451801
1,8844,65000000,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,...,Jumanji,6.9,2413.0,NaN,NaN,"[Adventure, Family, Fantasy]","[en, fr]","[English, Français]",1995,4.043035
2,15602,0,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,...,Grumpier Old Men,6.5,92.0,119050.0,Grumpy Old Men Collection,"[Comedy, Romance]",[en],[English],1995,0.000000
3,31357,16000000,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,...,Waiting to Exhale,6.1,34.0,NaN,NaN,"[Comedy, Drama, Romance]",[en],[English],1995,5.090760
4,11862,0,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,...,Father of the Bride Part II,5.7,173.0,96871.0,Father of the Bride Collection,[Comedy],[en],[English],1995,inf


In [65]:
#guardamos este datframe unido, esto es una parte, ya que falta verificar otros dataframes
merged_df.to_csv('..\datasets_crudos_CSV\datasetunido.csv')

In [ ]:
keys_list = []  # Lista vacía para almacenar las claves únicas

for index, row in df_movies.iterrows():
    countries_str = row['production_countries']
    
    if pd.notnull(countries_str):  # Verificar si el valor no es nulo
        countries_list = ast.literal_eval(countries_str)
        
        if isinstance(countries_list, list) and len(countries_list) > 0:
            keys = countries_list[0].keys()
            keys_list.extend(keys)

keys_list = list(set(keys_list))  # Eliminar duplicados

print(keys_list)


In [92]:
keys_list = []  # Lista vacía para almacenar las claves únicas

for index, row in df_movies.iterrows():
    countries_str = row['production_companies']
    
    if pd.notnull(countries_str):  # Verificar si el valor no es nulo
        countries_list = ast.literal_eval(countries_str)
        
        if isinstance(countries_list, list) and len(countries_list) > 0:
            keys = countries_list[0].keys()
            keys_list.extend(keys)

keys_list = list(set(keys_list))  # Eliminar duplicados

print(keys_list)


['name', 'id']


In [67]:
null_counts = merged_df.isnull().sum()
print(null_counts)


movies_id                   0
budget                      0
original_language          11
overview                  941
popularity                  0
production_companies        0
production_countries        0
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 25092
title                       0
vote_average                0
vote_count                  0
collection_id           41062
collection_name         41062
genres                      1
iso_639_1                   1
name                        1
release_year                0
return                      0
dtype: int64


In [68]:
##realizamos una pequeña prueba de busqueda
movie_id = 862 # Reemplaza 123 con el ID de la película que deseas buscar

movie_data = merged_df.loc[merged_df['movies_id'] == movie_id]
if not movie_data.empty:
    print(movie_data)
else:
    print("No se encontró ninguna película con el ID especificado.")

   movies_id    budget original_language  \
0        862  30000000                en   

                                            overview popularity  \
0  Led by Woody, Andy's toys live happily in his ...  21.946943   

                             production_companies  \
0  [{'name': 'Pixar Animation Studios', 'id': 3}]   

                                production_countries release_date  \
0  [{'iso_3166_1': 'US', 'name': 'United States o...   1995-10-30   

       revenue  runtime  ...      title vote_average vote_count  \
0  373554033.0     81.0  ...  Toy Story          7.7     5415.0   

   collection_id       collection_name                       genres iso_639_1  \
0        10194.0  Toy Story Collection  [Animation, Comedy, Family]      [en]   

        name release_year     return  
0  [English]         1995  12.451801  

[1 rows x 22 columns]


In [120]:
df_movies.to_csv('..\datasets_crudos_CSV\movies_prueba.csv')

# Cargamos los datos del DATASET CREDITS

In [69]:
credits = pd.read_csv('..\datasets_crudos_CSV\credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [70]:
#renombramos la columna id por movies_id
credits.rename(columns={'id': 'movies_id'}, inplace=True)
credits.head()

,cast,crew,movies_id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
# '''Creamos un función para hacer una prueba de buscar el contenido de las columnas
# buscando mendiante el id'''
# def buscar_contenido(id):
#     # Filtrar las filas con el ID especificado
#     filtro = credits['id'] == id
#     resultado = credits[filtro]
    
#     if len(resultado) == 0:
#         print(f"No se encontró ningún registro con el ID {id}")
#     else:
#         # Obtener el contenido del campo "Cast" y "Crew"
#         cast = resultado['cast'].iloc[0]
#         crew = resultado['crew'].iloc[0]
        
#         print(f"Contenido del campo 'Cast' para el ID {id}:")
#         print(cast)
#         print()
        
#         print(f"Contenido del campo 'Crew' para el ID {id}:")
#         print(crew)




In [71]:
'''Hemos observado que el dataset credits, contiene 3 columnas: id, cast, crew
pero cast y crew contiene diccionarios, lo que vamos a realizar es identificar las claves valores,
para luego realizar la separación de los datos'''
keys_list = [] # creamos una lista vacía

for row in credits['cast']: # iterar cada diccionario identificando la clave valor
    try:
        cast_list = ast.literal_eval(row)
        if isinstance(cast_list, list) and len(cast_list) > 0:
            keys = cast_list[0].keys()
            keys_list.extend(keys)
    except (SyntaxError, ValueError):
        pass

keys_list = list(set(keys_list))  # Eliminar duplicados

print(keys_list)


['order', 'name', 'cast_id', 'profile_path', 'gender', 'id', 'character', 'credit_id']


In [72]:
'''Despues de identificar las claves valores, vamos a proceder a desanidar los datos
para luego guardarlos a un nuevo dataframe'''
# Obtener las claves de los diccionarios
keys = ['id', 'cast_id', 'profile_path', 'credit_id', 'name', 'order', 'gender', 'character']

# Crear un DataFrame vacío con las columnas correspondientes a las claves y "movies_id"
new_df_cast = pd.DataFrame(columns=keys + ['movies_id'])

# Iterar sobre cada fila del dataframe "credits"
for index, row in credits.iterrows():
    try:
        cast_list = ast.literal_eval(row['cast'])
        if isinstance(cast_list, list) and len(cast_list) > 0:
            # Crear un diccionario temporal para almacenar los valores de cada clave
            temp_dict = {}
            
            # Obtener los valores de las claves para cada diccionario en la lista
            for key in keys:
                temp_dict[key] = [cast[key] if key in cast else None for cast in cast_list]
            
            # Agregar el valor de "movies_id" en cada fila
            temp_dict['movies_id'] = [row['movies_id']] * len(cast_list)
            
            # Convertir el diccionario temporal en un DataFrame y agregarlo al DataFrame final
            temp_df = pd.DataFrame(temp_dict)
            new_df_cast = pd.concat([new_df_cast, temp_df], ignore_index=True)
    except (SyntaxError, ValueError):
        pass


In [73]:

# Reordenar las columnas
new_df_cast = new_df_cast.reindex(columns=['movies_id', 'id', 'cast_id', 'profile_path', 'credit_id', 'name', 'order', 'gender', 'character'])
# Verificar el nuevo DataFrame resultante
new_df_cast.head()

,movies_id,id,cast_id,profile_path,credit_id,name,order,gender,character
0,862,31,14,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,52fe4284c3a36847f8024f95,Tom Hanks,0,2,Woody (voice)
1,862,12898,15,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,52fe4284c3a36847f8024f99,Tim Allen,1,2,Buzz Lightyear (voice)
2,862,7167,16,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,52fe4284c3a36847f8024f9d,Don Rickles,2,2,Mr. Potato Head (voice)
3,862,12899,17,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,52fe4284c3a36847f8024fa1,Jim Varney,3,2,Slinky Dog (voice)
4,862,12900,18,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,52fe4284c3a36847f8024fa5,Wallace Shawn,4,2,Rex (voice)


In [74]:
#guardamos nuestro dataframe con el nombre de cast.csv
new_df_cast.to_csv('../datasets_crudos_CSV/cast.csv', index=False)

In [75]:
# Obtener la cantidad de filas y columnas del dataframe cast
num_rows = new_df_cast.shape[0]
num_cols = new_df_cast.shape[1]

# Imprimir la cantidad total de filas y columnas
print("Cantidad de filas:", num_rows)
print("Cantidad de columnas:", num_cols)

Cantidad de filas: 562474
Cantidad de columnas: 9


In [76]:
'''Despues de realizar el proceso de obtener las claves valores de los diccionarios 
de la columna cast, aplicamos lo mismo para la columna crew'''
keys_list = [] #creamos una lista vacía

for row in credits['crew']:#iteramos para identificar las claves valores
    try:
        crew_list = ast.literal_eval(row)
        if isinstance(crew_list, list) and len(crew_list) > 0:
            keys = crew_list[0].keys()
            keys_list.extend(keys)
    except (SyntaxError, ValueError):
        pass

keys_list = list(set(keys_list))  # Eliminar duplicados

print(keys_list)

['department', 'name', 'profile_path', 'job', 'gender', 'id', 'credit_id']


In [77]:
'''Ahora procedemos a realizar los pasos para desanidar los datos, ahora de la columna crew
y despues guardarlos a un dataframe nuevo'''

# Obtener las claves de los diccionarios
keys = ['id', 'department', 'profile_path', 'credit_id', 'name', 'gender', 'job']

# Crear un DataFrame vacío con las columnas correspondientes a las claves
new_df_crew = pd.DataFrame(columns=keys + ['movies_id'])

# Iterar sobre cada fila de la columna "crew" y separar los datos en columnas
for index, row in credits.iterrows():
    try:
        crew_list = ast.literal_eval(row['crew'])
        if isinstance(crew_list, list) and len(crew_list) > 0:
            # Crear un diccionario temporal para almacenar los valores de cada clave
            temp_dict = {}
            
            # Obtener los valores de las claves para cada diccionario en la lista
            for key in keys:
                temp_dict[key] = [crew[key] if key in crew else None for crew in crew_list]
            
            # Agregar el valor de movies_id para cada fila
            temp_dict['movies_id'] = [row['movies_id']] * len(crew_list)
            
            # Convertir el diccionario temporal en un DataFrame y agregarlo al DataFrame final
            temp_df = pd.DataFrame(temp_dict)
            new_df_crew = pd.concat([new_df_crew, temp_df], ignore_index=True)
    except (SyntaxError, ValueError):
        pass



In [78]:
# Verificar el resultado
new_df_crew.head()

,id,department,profile_path,credit_id,name,gender,job,movies_id
0,7879,Directing,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,52fe4284c3a36847f8024f49,John Lasseter,2,Director,862
1,12891,Writing,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,52fe4284c3a36847f8024f4f,Joss Whedon,2,Screenplay,862
2,7,Writing,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,52fe4284c3a36847f8024f55,Andrew Stanton,2,Screenplay,862
3,12892,Writing,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,52fe4284c3a36847f8024f5b,Joel Cohen,2,Screenplay,862
4,12893,Writing,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,52fe4284c3a36847f8024f61,Alec Sokolow,0,Screenplay,862


In [79]:
# Obtener la cantidad de filas y columnas crew
num_rows = new_df_crew.shape[0]
num_cols = new_df_crew.shape[1]

# Imprimir la cantidad total de filas y columnas
print("Cantidad de filas:", num_rows)
print("Cantidad de columnas:", num_cols)

Cantidad de filas: 464314
Cantidad de columnas: 8


In [80]:
#guardamos nuestro dataframe como crew.csv
new_df_crew.to_csv('../datasets_crudos_CSV/crew.csv', index=False)

In [81]:
# Fusionar los dataframes utilizando la columna "movies_id" como clave de fusión
combined_df = new_df_crew.merge(new_df_cast, on='movies_id', how='outer')



C:\Users\Ricky Quinteros\AppData\Local\Temp\ipykernel_11040\3892713818.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  combined_df = new_df_crew.merge(new_df_cast, on='movies_id', how='outer')


In [82]:
#eliminar las columnas "profile_path_x" y "profile_path_y" del dataframe combined_df
combined_df = combined_df.drop(['profile_path_x', 'profile_path_y'], axis=1)


In [83]:

# Verificar el resultado
combined_df.head()


,id_x,department,credit_id_x,name_x,gender_x,job,movies_id,id_y,cast_id,credit_id_y,name_y,order,gender_y,character
0,7879,Directing,52fe4284c3a36847f8024f49,John Lasseter,2,Director,862.0,31,14,52fe4284c3a36847f8024f95,Tom Hanks,0,2,Woody (voice)
1,7879,Directing,52fe4284c3a36847f8024f49,John Lasseter,2,Director,862.0,12898,15,52fe4284c3a36847f8024f99,Tim Allen,1,2,Buzz Lightyear (voice)
2,7879,Directing,52fe4284c3a36847f8024f49,John Lasseter,2,Director,862.0,7167,16,52fe4284c3a36847f8024f9d,Don Rickles,2,2,Mr. Potato Head (voice)
3,7879,Directing,52fe4284c3a36847f8024f49,John Lasseter,2,Director,862.0,12899,17,52fe4284c3a36847f8024fa1,Jim Varney,3,2,Slinky Dog (voice)
4,7879,Directing,52fe4284c3a36847f8024f49,John Lasseter,2,Director,862.0,12900,18,52fe4284c3a36847f8024fa5,Wallace Shawn,4,2,Rex (voice)


In [84]:
#renombrar las columnas en el dataframe combined_df
combined_df = combined_df.rename(columns={'name_x': 'name_crew',
                                          'name_y': 'name_cast',
                                          'gender_x': 'gender_crew',
                                          'gender_y': 'gender_cast',
                                          'character': 'character_cast'})


In [85]:
#ordenamos nuestras columnas
column_order = ['movies_id', 'id_x', 'department', 'credit_id_x', 'name_crew', 'gender_crew', 'job',
                'id_y', 'cast_id', 'credit_id_y', 'name_cast', 'order', 'gender_cast', 'character_cast']

combined_df = combined_df.reindex(columns=column_order)



In [86]:
#eliminamos estas columnas
columns_to_drop = ['id_x', 'credit_id_x', 'id_y', 'cast_id', 'credit_id_y']
combined_df = combined_df.drop(columns=columns_to_drop)

In [87]:
combined_df.head(2)

,movies_id,department,name_crew,gender_crew,job,name_cast,order,gender_cast,character_cast
0,862.0,Directing,John Lasseter,2,Director,Tom Hanks,0,2,Woody (voice)
1,862.0,Directing,John Lasseter,2,Director,Tim Allen,1,2,Buzz Lightyear (voice)


In [88]:
#una prueba
movie_id = 862  # Reemplaza 12345 con el movies_id de la película que deseas seleccionar

selected_movie = combined_df.loc[combined_df['movies_id'] == movie_id]
print(selected_movie)


     movies_id  department       name_crew gender_crew                job  \
0        862.0   Directing   John Lasseter           2           Director   
1        862.0   Directing   John Lasseter           2           Director   
2        862.0   Directing   John Lasseter           2           Director   
3        862.0   Directing   John Lasseter           2           Director   
4        862.0   Directing   John Lasseter           2           Director   
...        ...         ...             ...         ...                ...   
1373     862.0  Production  Mickie McGowan           0  ADR Voice Casting   
1374     862.0  Production  Mickie McGowan           0  ADR Voice Casting   
1375     862.0  Production  Mickie McGowan           0  ADR Voice Casting   
1376     862.0  Production  Mickie McGowan           0  ADR Voice Casting   
1377     862.0  Production  Mickie McGowan           0  ADR Voice Casting   

            name_cast order gender_cast           character_cast  
0       

In [89]:
##guardamos el dataframe
combined_df.to_csv('../datasets_crudos_CSV/credits_final.csv')

In [90]:
merged_df.head(2)

,movies_id,budget,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,title,vote_average,vote_count,collection_id,collection_name,genres,iso_639_1,name,release_year,return
0,862,30000000,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,...,Toy Story,7.7,5415.0,10194.0,Toy Story Collection,"[Animation, Comedy, Family]",[en],[English],1995,12.451801
1,8844,65000000,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,...,Jumanji,6.9,2413.0,NaN,NaN,"[Adventure, Family, Fantasy]","[en, fr]","[English, Français]",1995,4.043035


In [112]:
# Verificar los valores nulos en merged_df
null_counts = merged_df.isnull().sum()

# Imprimir los valores nulos por columna
print(null_counts)

movies_id               0
budget                  0
original_language       0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
status                  0
title                   0
vote_average            0
vote_count              0
genres                  1
iso_639_1               1
name                    1
release_year            0
return                  0
dtype: int64


'''verificamos los valores vacios de overview, ya que se observa que contiene una pequeña sipnosis
de la pelicula, si eliminamos la filas vacias puede que se pierdad información así que reemplazaremos por
sipnosis no disponible'''

In [94]:
# Explorar los valores únicos en la columna 'overview'
unique_overviews = merged_df['overview'].unique()
print(unique_overviews)


["Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."
 "When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."
 "A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is

In [103]:
# Reemplazar los valores vacíos en la columna 'overview' con "Sin sinopsis disponible"
merged_df['overview'] = merged_df['overview'].fillna("Sin sinopsis disponible")

'''en la columna de lenguajes se observa que cada pelicula tiene un idioma en la cual esta 
originado, para no perder datos, los espacios vaciós los reemplazamos por unknow'''

In [93]:
# Explorar los valores únicos en la columna 'original_language'
unique_languages = merged_df['original_language'].unique()
print(unique_languages)

['en' 'fr' 'zh' 'it' 'fa' 'nl' 'de' 'cn' 'ar' 'es' 'ru' 'sv' 'ja' 'ko'
 'sr' 'bn' 'he' 'pt' 'wo' 'ro' 'hu' 'cy' 'vi' 'cs' 'da' 'no' 'nb' 'pl'
 'el' 'sh' 'xx' 'mk' 'bo' 'ca' 'fi' 'th' 'sk' 'bs' 'hi' 'tr' 'is' 'ps'
 'ab' 'eo' 'ka' 'mn' 'bm' 'zu' 'uk' 'af' 'la' 'et' 'ku' 'fy' 'lv' 'ta'
 'sl' 'tl' 'ur' 'rw' 'id' 'bg' 'mr' 'lt' 'kk' 'ms' 'sq' nan 'qu' 'te' 'am'
 'jv' 'tg' 'ml' 'hr' 'lo' 'ay' 'kn' 'eu' 'ne' 'pa' 'ky' 'gl' 'uz' 'sm'
 'mt' 'hy' 'iu' 'lb' 'si']


In [104]:

# Reemplazar los valores vacíos en la columna 'original_language' con un valor específico, por ejemplo 'Unknown'
merged_df['original_language'] = merged_df['original_language'].fillna('Unknown')


'''En el caso de los valores de la columna runtime son datos numericos, vamos a reemplazar los valores nulos por el promedio para evitar que se pierdan datos de la data'''

In [95]:
# Explorar los valores únicos en la columna 'runtime'
unique_runtimes = merged_df['runtime'].unique()
print(unique_runtimes)

[8.100e+01 1.040e+02 1.010e+02 1.270e+02 1.060e+02 1.700e+02 9.700e+01
 1.300e+02 8.800e+01 7.800e+01 1.920e+02 1.190e+02 1.780e+02 1.360e+02
 9.800e+01 9.000e+01 1.030e+02 1.050e+02 1.240e+02 1.320e+02 1.110e+02
 1.120e+02 1.230e+02 1.000e+02 1.080e+02 9.900e+01 1.290e+02 5.000e+01
 8.900e+01 1.210e+02 1.220e+02 5.100e+01 1.170e+02 1.160e+02 9.600e+01
 9.300e+01 9.500e+01 1.150e+02 1.370e+02 9.200e+01 8.500e+01 9.100e+01
 1.750e+02 8.700e+01 7.000e+01 1.180e+02 1.020e+02 1.350e+02 1.100e+02
 1.770e+02 1.140e+02 1.130e+02 8.600e+01 1.250e+02 1.400e+02 1.390e+02
 8.200e+01 8.300e+01 1.280e+02 1.090e+02 8.000e+01 1.340e+02 1.070e+02
 9.400e+01 1.450e+02 7.300e+01 0.000e+00 7.700e+01 1.710e+02 1.330e+02
 1.540e+02 1.620e+02 1.420e+02 1.260e+02 1.410e+02 1.800e+02 1.910e+02
 6.100e+01 8.400e+01 1.440e+02 1.310e+02 1.380e+02 1.950e+02 1.870e+02
 7.600e+01 1.200e+02 1.600e+02 1.810e+02 6.000e+01       nan 7.900e+01
 2.500e+01 7.500e+01 7.400e+01 3.000e+01 1.490e+02 7.100e+01 2.380e+02
 2.010

In [106]:
# Calcular el promedio de los valores no nulos en la columna 'runtime'
average_runtime = merged_df['runtime'].mean()

# Reemplazar los valores nulos en la columna 'runtime' con el promedio
merged_df['runtime'] = merged_df['runtime'].fillna(average_runtime)

'''En el caso de la columna status, no hay mucha informacion para poder reemplazar adecuadamente en que estado son estan las peliculas que no tienen valores, asi que vamos a colocar como  valor desconocido'''

In [96]:
# Explorar los valores únicos en la columna 'status'
unique_statuses = merged_df['status'].unique()
print(unique_statuses)

['Released' nan 'Rumored' 'Post Production' 'In Production' 'Planned'
 'Canceled']


In [107]:
merged_df['status'] = merged_df['status'].fillna('Desconocido')

'''En el caso de las columnas  'collection_id' y 'collection_name' ' tagline' los valores nulos son significativos
por lo cual vamos a eliminar esas columnas

In [102]:
# Explorar los valores únicos en la columna 'tagline'
unique_taglines = merged_df['tagline'].unique()
print(unique_taglines)

[nan 'Roll the dice and unleash the excitement!'
 'Still Yelling. Still Fighting. Still Ready for Love.' ...
 'Meet...The CREEPER!'
 'Do you know what happened 50 years before "The Blair Witch Project"?'
 'A deadly game of wits.']


In [ ]:
#Eliminamos las columnas 'collection_id', 'collection_name', 'tagline'
merged_df = merged_df.drop(['collection_id', 'collection_name', 'tagline'], axis=1)

In [114]:
#guardamos nuestro dataset
merged_df.to_csv('../datasets_crudos_CSV/credits_final.csv')

In [111]:
combined_df.head(2)

,movies_id,department,name_crew,gender_crew,job,name_cast,order,gender_cast,character_cast
0,862.0,Directing,John Lasseter,2,Director,Tom Hanks,0,2,Woody (voice)
1,862.0,Directing,John Lasseter,2,Director,Tim Allen,1,2,Buzz Lightyear (voice)
